In [1]:
import sys
import torch
sys.path.append("../src")
from interpret import Abs_interpreter
from domain import Interval

In [2]:
# Class to handle interval domain operations
class IntervalDomainHandler():
    def add(self, i1, i2):
        return (i1[0]+i2[0], i1[1]+i2[1])
    
    def subtract(self, i1, i2):
        return (i1[0]-i2[1], i1[1]-i2[0])
    
    def mul(self, i1, i2):
        op1 = i1[0] * i2[0]
        op2 = i1[0] * i2[1]
        op3 = i1[1] * i2[0]
        op4 = i1[1] * i2[1]
        
        return (min(op1, op2, op3, op4), max(op1, op2, op3, op4))

In [3]:
def compute_pinn_burgers_residual_bounds(model_name, inp_range = (0,1), abstract_domain = 'Interval'):
    """
    Takes in a pinn burgers model, input range and the abstract domain.
    Computes the residual bound of the model on that input range using the specified abstract domain.
    """
    model = torch.load(model_name)
    
    if abstract_domain == 'Interval':
        domain = Interval()
    else:
        raise NotImplementedError("Unknown abstract domain: " + domain)
        
    abs_interpreter = Abs_interpreter(model, domain)
    val_bounds, deriv_bounds = abs_interpreter.forward_pass(inp_range[0], inp_range[1])

    ul = val_bounds[0][0][0].item()
    ur = val_bounds[1][0][0].item()
    
    uxl = deriv_bounds[0][0][0].item()
    utl = deriv_bounds[0][0][1].item()
    
    uxr = deriv_bounds[1][0][0].item()
    utr = deriv_bounds[1][0][1].item()
        
    interval_handler = IntervalDomainHandler()
    
    # ut + u*ux
    residual_bounds = interval_handler.add((utl, utr), interval_handler.mul((ul, ur), (uxl, uxr)))
    return residual_bounds

# Bounds calculated on the complete domain (without input splitting)
compute_pinn_burgers_residual_bounds("../trained_models/pinn-burgers.pt")

/Users/sgomber/Desktop/Projects/sciml_proj/experiments/../src/domain.py:108: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1773.)
  U_tanh[indices] = 1


(-2430.2961955345236, 2294.239749767119)

## Computing bounds using input splitting

In [6]:
def compute_pinn_bounds_using_input_splitting(model_name, num_partitions, inp_range = (0,1), abstract_domain = 'Interval'):
    """
    Compute pinn_burger bounds on the specified input range by partitioning into the specified
    number of partitions.
    """
    inp_low = inp_range[0]
    inp_high = inp_range[1]
    eps = (inp_high - inp_low)/num_partitions

    l_final = None
    u_final = None

    for p in range(num_partitions):
        bounds = compute_pinn_burgers_residual_bounds(model_name, inp_range=(inp_low + eps*p, inp_low + eps*(p+1)), abstract_domain = abstract_domain)

        if l_final is None: l_final = bounds[0]
        if u_final is None: u_final = bounds[1]

        l_final = min(l_final, bounds[0])
        u_final = max(u_final, bounds[1])

    return l_final, u_final

In [11]:
print(compute_pinn_bounds_using_input_splitting("../trained_models/pinn-burgers.pt", 2500))

(-0.7570360751746037, 0.1805245778269864)


In [12]:
print(compute_pinn_bounds_using_input_splitting("../trained_models/pinn-burgers_weak.pt", 2500))

(-0.1232487267407194, 1.9427881401686022)
